<p float="center">
  <img src="images/horizontal.png" alt="Coiled logo" width="415" hspace="10"/>
</p>



<p float="center">
  <img src="images/dask_horizontal_no_pad.svg" alt="Dask logo" width="400" hspace="10" />
</p>

# Scalable Data Science

In this notebook, we'll 

* Perform a basic analytics and ETL workflow on the NYC taxi dataset using Pandas;
* Scale up this workflow to a dataset that doesn't fit in RAM using Dask;
* (Optional) Scale out this workflow to leverage a cluster on the Cloud using Coiled.

The workflow is intentionally boring so that we can see the power of scalable data science immediately: we'll load some data, convert some data types for more efficient storage, add a new column, perform some basic analytics, save the new DataFrame to file.

In the notebooks that follow, we'll jump into more interesting examples, including machine learning.

*A bit about me:* I'm Hugo Bowne-Anderson, Head of Data Science Evangelism and Marketing at [Coiled](coiled.io/). We build products that bring the power of scalable data science and machine learning to you, such as single-click hosted clusters on the cloud. We want to take the DevOps out of data science so you can get back to your real job. If you're interested in taking Coiled for a test drive, you can sign up for our [free Beta here](beta.coiled.io/).

Before scaling up, let's look at a common workflow in Pandas.

## 1. Pandas: Convert CSV to Parquet and Engineer a Feature

<img src="images/pandas-logo.svg" alt="pandas logo" style="width: 500px;"/>

In the following, we'll 

* use Pandas to load in part of the NYC taxi dataset from a CSV, 
* massage the data, 
* engineer a feature, 
* compute the average tip as a function of the number of passengers, and 
* save to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable).

If you're following along in Binder, you won't be able to execute the code but you can read it.

### Download the data from Amazon

In [ ]:
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-{01..12}.csv

In [1]:
# Check out head of 1st file
!head data_taxi/yellow_tripdata_2019-01.csv

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7,0.5,0.5,1.65,0,0.3,9.95,
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14,0.5,0.5,1,0,0.3,16.3,
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,.00,1,N,236,236,1,4.5,0.5,0.5,0,0,0.3,5.8,
2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,.00,1,N,193,193,2,3.5,0.5,0.5,0,0,0.3,7.55,
2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,.00,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,
2,2018-11-28 16:25:49,2018-11-28 16:28:26,5,.00,1,N,193,193,2,3.5,0.5,0.5,0,5.76,0.3,13.31,
2,2018-11-28 16:29:37,2018-11-28 16:33:43,5,.00,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,
1,2019-01-01 00:21:28,2019-01-01 00:28:37,1,1.30,1,N,163,229,1,6.5,0.5,0.5,1.25,0,0.3,9.05,
1,2019-01-01 00:32:01,2019-01-01 0

Note: this will take at least several minutes to download the above.

### Investigate data locally with Pandas


In [3]:
%%time
# Import pandas and read in beginning of 1st file
import pandas as pd
df = pd.read_csv(
    "data_taxi/yellow_tripdata_2019-01.csv", 
    #nrows=10000,
)
df

CPU times: user 15.9 s, sys: 4.69 s, total: 20.6 s
Wall time: 22 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.00,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.00,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.00,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7667787,2,2019-01-31 23:57:36,2019-02-01 00:18:39,1,4.79,1,N,263,4,1,18.0,0.5,0.5,3.86,0.0,0.3,23.16,0.0
7667788,2,2019-01-31 23:32:03,2019-01-31 23:33:11,1,0.00,1,N,193,193,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
7667789,2,2019-01-31 23:36:36,2019-01-31 23:36:40,1,0.00,1,N,264,264,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
7667790,2,2019-01-31 23:14:53,2019-01-31 23:15:20,1,0.00,1,N,264,7,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [4]:
df.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
RatecodeID                 int64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

### Massage data types 

Before we convert to parquet format, let's clean up our types a little

In [5]:
%%time

# Read in data, parse dates
df = pd.read_csv(
    "data_taxi/yellow_tripdata_2019-01.csv", 
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
)
df

CPU times: user 17.3 s, sys: 3.61 s, total: 20.9 s
Wall time: 21.1 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.00,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.00,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.00,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7667787,2,2019-01-31 23:57:36,2019-02-01 00:18:39,1,4.79,1,N,263,4,1,18.0,0.5,0.5,3.86,0.0,0.3,23.16,0.0
7667788,2,2019-01-31 23:32:03,2019-01-31 23:33:11,1,0.00,1,N,193,193,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
7667789,2,2019-01-31 23:36:36,2019-01-31 23:36:40,1,0.00,1,N,264,264,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
7667790,2,2019-01-31 23:14:53,2019-01-31 23:15:20,1,0.00,1,N,264,7,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0


In [6]:
%%time

# Alter data types for efficiency
df = df.astype({
    "VendorID": "uint8",
    "passenger_count": "uint8",
    "RatecodeID": "uint8",
    "store_and_fwd_flag": "category",
    "PULocationID": "uint16",
    "DOLocationID": "uint16",    
})

# Create new feature in dataset: tip_ratio
df["tip_ratio"] = df.tip_amount / df.total_amount

CPU times: user 1.41 s, sys: 2.1 s, total: 3.51 s
Wall time: 2.81 s


## Basic Analytics

In [ ]:
%%time

# Compute average tip as a function of the number of passengers
df.groupby("passenger_count").tip_amount.mean()

### Convert to Parquet

In [ ]:
%%time
df.to_parquet("data_taxi/yellow_tripdata_2019-01.parq")

In [ ]:
%%time

# Read from parquet and time it
df = pd.read_parquet("data_taxi/yellow_tripdata_2019-01.parq")
df

**Recap:** We have

* used Pandas to load in part of the NYC taxi dataset from a CSV, 
* massaged the data, 
* engineered a feature, 
* computed the average tip as a function of the number of passengers, and 
* saved to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable).

### Operate on many files in a for loop?

We could do this, but it's unpleasant

```python
for filename in glob("~/data/nyctaxi/yellow_tripdata_2019-*.parq"):
    df = pd.read_csv(filename)
    ...
    df.to_parquet(...)
```

## 2. Use Dask locally to process the full dataset

<img src="images/dask_horizontal_no_pad.svg" alt="Dask logo" style="width: 500px;"/>

The full NYC taxi dataset won't even fit in the RAM of my laptop. Do I need a large or external cluster yet? No. First, I can take advantage of all the cores on my laptop in parallel. This is what we call *scaling up* our computation (out-of-core computing). Later we'll see how to *scale out* computation across a cluster.

One way of doing this is with [Dask](dask.org/). As we're about to see, part of the value of Dask lies in its API being as close as possible to the PyData APIs we know and love, in this case, Pandas.

In [the words of Matthew Rocklin](https://coiled.io/blog/history-dask/), core developer and co-maintainer of Dask and CEO of Coiled, there was a social goal of Dask:
> Invent nothing. We wanted to be as familiar as possible to what users already knew in the PyData stack

Let's do it!

The plan:

* use Dask to load in **all** of the NYC taxi dataset from 10+ CSVs (8+ GBs), 
* massage the data, 
* engineer a feature,
* compute the average tip as a function of the number of passengers, and
* save to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable).

We'll also dive into the basics of Dask and distributed compute (but we'll execute some code first and dive into this part while it runs!).

In [ ]:
# Import Dask parts, spin up a local cluster, and instantiate a Client
from dask.distributed import LocalCluster, Client
cluster = LocalCluster(n_workers=4)
client = Client(cluster)
client

In [ ]:
%%time

import dask.dataframe as dd

# Import the full dataset (note the Dask API!)
df = dd.read_csv(
    "data_taxi/yellow_tripdata_2019-*.csv", 
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={'RatecodeID': 'float64',
       'VendorID': 'float64',
       'passenger_count': 'float64',
       'payment_type': 'float64'}

)
df

In [ ]:
# Alter data types for efficiency
df = df.astype({
    "VendorID": "UInt8",
    "passenger_count": "UInt8",
    "RatecodeID": "UInt8",
    "store_and_fwd_flag": "category",
    "PULocationID": "UInt16",
    "DOLocationID": "UInt16",    
})

# Create new feature in dataset: tip_ratio
df["tip_ratio"] = df.tip_amount / df.total_amount

In [ ]:
%%time

# Prepare to compute the average tip 
# as a function of the number of passengers
mean_amount = df.groupby("passenger_count").tip_amount.mean()

In [ ]:
%%time

# compute the average tip as a function of the number of passengers
mean_amount.compute()

In [ ]:
%%time
df.to_parquet("data_taxi/yellow_tripdata_2019.parq")

### 2a Notes on what is happening in Dask and Python

The above code will take some time to run so let's take this opportunity to see what is going on with Dask, Python, and the distributed computation.

#### Components of Dask

Dask contains 3 main components and we have already see two of them above:
* High-level collections in the form of Dask DataFrames (these set up the step of your computation);
* Schedulers (these actually execute the computation, in this case, on a single machine).

Let's get a sense for what these are.

<img src="images/dask-components.svg" width="400px">

#### Dask DataFrames

What exactly is this Dask DataFrame? A schematic is worth a thousand words:

<img src="images/dask-dataframe.svg" width="400px">

Essentially, the Dask DataFrame is a large, virtual dataframe divided along the index into multiple Pandas DataFrames.

#### Dask Schedulers, Workers, and beyond

<img src="images/dask-cluster.svg" width="400px">

Work (Python code) is performed on a cluster, which consists of 
* a _scheduler_ (which manages and sends the work / tasks to the workers)
* _workers_, which compute the tasks.

The _client_ is "the user-facing entry point for cluster users." What this means is that the client lives wherever you are writing your Python code and the client talks to the scheduler, passing it the tasks.

**Recap:** We have

* used Dask to load in **all** of the NYC taxi dataset from 10+ CSVs (8+ GBs), 
* massaged the data, 
* engineered a feature, 
* computed the average tip as a function of the number of passengers, and 
* saved to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable),
* dived into the basic of Dask and distributed compute and understand the basic concepts.

## 3. Optional: Work directly from the cloud with Coiled 

<br>
<img src="images/horizontal.png" alt="Coiled logo" style="width: 500px;"/>
<br>

Here I'll spin up a cluster on Coiled to show you just how easy it can be. Note that to do so, I've also signed into the [Coiled Beta](beta.coiled.io/), pip installed `coiled`, and authenticated. You can do the same!

The plan:

* use Coiled to load in **all** of the NYC taxi dataset from 10+ CSVs (8+ GBs) on an AWS cluster, 
* massage the data, 
* engineer a feature, 
* compute the average tip as a function of the number of passengers, and 
* save to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable).

In [ ]:
import coiled
from dask.distributed import LocalCluster, Client

In [ ]:
# Create a Software Environment
coiled.create_software_environment(
    name="my-software-env",
    conda="binder/environment.yml",
)

In [ ]:
# Control the resources of your cluster by creating a new cluster configuration
coiled.create_cluster_configuration(
    name="my-cluster-config",
    worker_memory="16 GiB",
    worker_cpu=4,
    scheduler_memory="8 GiB",
    scheduler_cpu=2,
    software="my-software-env",
)


In [ ]:
# Spin up cluster, instantiate a Client
cluster = coiled.Cluster(n_workers=10, configuration="my-cluster-config")
client = Client(cluster)
client

In [ ]:
import dask.dataframe as dd

# Read data into a Dask DataFrame
df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv", 
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={
        'RatecodeID': 'float64',
       'VendorID': 'float64',
       'passenger_count': 'float64',
       'payment_type': 'float64'
    },
    storage_options={"anon":True}
)
df

In [ ]:
%%time

# Prepare to compute the average tip 
# as a function of the number of passengers
mean_amount = df.groupby("passenger_count").tip_amount.mean()

In [ ]:
%%time

# Compute the average tip 
# as a function of the number of passengers
mean_amount.compute()

In [ ]:
# Alter data types for efficiency
df = df.astype({
    "VendorID": "UInt8",
    "passenger_count": "UInt8",
    "RatecodeID": "UInt8",
    "store_and_fwd_flag": "category",
    "PULocationID": "UInt16",
    "DOLocationID": "UInt16",    
})

# Create new feature in dataset: tip_ratio
df["tip_ratio"] = df.tip_amount / df.total_amount

In [ ]:
%%time
df.to_parquet("s3://hugo-coiled-tutorial/nyctaxi-2019.parq")

In [ ]:
client.shutdown()

**Recap:** We have
* used Coiled to load in **all** of the NYC taxi dataset from 10+ CSVs (10 GBs) on an AWS cluster, 
* massaged the data, 
* engineered a feature, 
* computed the average tip as a function of the number of passengers, and 
* saved to [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) (far more efficient than CSV, but not human-readable).